In [1]:
import datetime, warnings, scipy 
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt

plt.rcParams["patch.force_edgecolor"] = True
plt.style.use('fivethirtyeight')
mpl.rc('patch', edgecolor='dimgray', linewidth=1)

from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "last_expr"
pd.options.display.max_columns = 50
%matplotlib inline
warnings.filterwarnings("ignore")
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error

In [7]:
df = pd.read_csv('india-air-quality-2015-2020/Delhi/aqi_calculated.csv')
df

,Datetime,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3,Benzene,Toluene,Xylene,PM10_24hr_avg,PM2.5_24hr_avg,SO2_24hr_avg,NOx_24hr_avg,NH3_24hr_avg,CO_8hr_max,O3_8hr_max,PM2.5_SubIndex,PM10_SubIndex,SO2_SubIndex,NOx_SubIndex,NH3_SubIndex,CO_SubIndex,O3_SubIndex,AQI_bucket_calculated
0,01/01/2015 16:00,211.51,340.66,13.09,37.08,47.07,33.54,15.24,12.33,34.35,3.51,5.85,4.73,681.438437,323.240000,6.288750,93.116875,30.975000,24.15,55.49,456.338461,714.298047,7.860938,113.116875,7.743750,342.058824,55.49,Severe
1,01/01/2015 17:00,191.12,257.40,15.44,38.40,51.24,34.98,13.11,18.00,34.38,3.58,5.23,3.78,656.495000,315.468235,6.977647,90.653529,31.210588,24.15,55.49,450.360181,683.118750,8.722059,110.653529,7.802647,342.058824,55.49,Severe
2,01/01/2015 18:00,218.44,351.01,22.85,44.73,59.26,34.80,18.35,19.17,38.42,3.67,5.93,4.98,639.523611,310.077778,7.655000,88.909444,31.410000,24.15,55.49,446.213675,661.904514,9.568750,108.909444,7.852500,342.058824,55.49,Severe
3,01/01/2015 19:00,296.80,600.95,69.20,47.76,101.68,34.25,16.67,21.50,49.12,14.75,27.13,8.78,637.493421,309.378947,8.383684,89.581579,31.559474,24.15,53.17,445.676113,659.366776,10.479605,109.581579,7.889868,342.058824,53.17,Severe
4,01/01/2015 20:00,336.43,714.63,148.42,46.31,171.10,35.75,12.17,17.67,56.44,26.56,45.62,9.99,641.350250,310.731500,8.848000,93.657500,31.769000,24.15,56.44,446.716539,664.187812,11.060000,113.657500,7.942250,342.058824,56.44,Severe
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48172,06/30/2020 20:00,43.19,123.26,6.33,22.28,17.66,29.92,0.92,10.64,44.13,1.63,17.83,0.86,84.418750,36.862500,10.532500,16.069167,28.139583,0.92,62.55,61.437500,84.418750,13.165625,20.086458,7.034896,46.000000,62.55,Satisfactory
48173,06/30/2020 21:00,45.30,125.22,6.34,22.91,17.96,29.95,1.04,10.59,39.62,1.72,18.06,1.16,87.082500,37.884583,10.499583,15.740000,28.276667,1.04,62.55,63.140972,87.082500,13.124479,19.675000,7.069167,52.000000,62.55,Satisfactory
48174,06/30/2020 22:00,49.04,129.39,6.29,20.95,16.80,30.01,1.00,10.20,38.95,1.66,16.17,1.18,89.670000,38.911667,10.477500,15.528750,28.374167,1.04,62.55,64.852778,89.670000,13.096875,19.410937,7.093542,52.000000,62.55,Satisfactory
48175,06/30/2020 23:00,50.51,128.86,6.19,20.40,16.83,29.40,0.94,10.98,34.53,1.60,16.79,0.85,91.976250,39.800000,10.508750,15.436250,28.479167,1.04,62.55,66.333333,91.976250,13.135938,19.295313,7.119792,52.000000,62.55,Satisfactory


In [10]:
df = df.drop(columns=['Datetime', 'Benzene', 'Toluene', 'Xylene', 'AQI_bucket_calculated'])

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48177 entries, 0 to 48176
Data columns (total 23 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   PM2.5           48177 non-null  float64
 1   PM10            48177 non-null  float64
 2   NO              48177 non-null  float64
 3   NO2             48177 non-null  float64
 4   NOx             48177 non-null  float64
 5   NH3             48177 non-null  float64
 6   CO              48177 non-null  float64
 7   SO2             48177 non-null  float64
 8   O3              48177 non-null  float64
 9   PM10_24hr_avg   48177 non-null  float64
 10  PM2.5_24hr_avg  48177 non-null  float64
 11  SO2_24hr_avg    48177 non-null  float64
 12  NOx_24hr_avg    48177 non-null  float64
 13  NH3_24hr_avg    48177 non-null  float64
 14  CO_8hr_max      48177 non-null  float64
 15  O3_8hr_max      48177 non-null  float64
 16  PM2.5_SubIndex  48177 non-null  float64
 17  PM10_SubIndex   48177 non-null 

In [ ]:
aqi = df[['AQI', 'AQI_Bucket']]
df = df.drop(columns=['AQI', 'AQI_Bucket'])
aqi.to_csv('india-air-quality-2015-2020/Delhi/aqi.csv', index=False)

In [ ]:
df

In [ ]:
missing = df.isnull().sum() / len(df) * 100

In [ ]:
type(missing)

In [ ]:
t = missing.to_dict()
# t.pop("Datetime")
# t.pop("Benzene")
# t.pop("Toluene")
# t.pop("Xylene")
t


In [ ]:
plt.bar(range(len(t)), list(t.values()), align='center')
plt.xticks(range(len(t)), list(t.keys()), rotation=90)

plt.title('Delhi missing values percentage')
plt.show()

In [ ]:
df.head(30)

In [ ]:
sns.boxplot(data=df)

#modify axis labels
plt.xlabel('Columns')
plt.xticks(rotation=90)
plt.ylabel('Value')
plt.show()

In [ ]:
df.plot(figsize=(20,8))

In [ ]:
for column in df.columns:
    if column != "Datetime":
        df[column] = df[column].interpolate(method="linear")

In [ ]:
# def convert_outlier(column):
#     q1 = column.quantile(0.25)
#     q3 = column.quantile(0.75)
#     iqr = q3 - q1
#     lower_bound = q1 - 1.5 * iqr
#     upper_bound = q3 + 1.5 * iqr
#     column[column < lower_bound] = np.nan
#     column[column > upper_bound] = np.nan
#     return column

# for column in df.columns:
#     if column != 'Datetime':
#         df[column] = convert_outlier(df[column])
#         df[column] = df[column].interpolate(method='linear')

In [ ]:
df.isnull().sum()

In [ ]:
sns.boxplot(data=df)

#modify axis labels
plt.xlabel('Columns')
plt.xticks(rotation=90)
plt.ylabel('Value')

plt.show()

In [ ]:
df.head(20)

In [ ]:
df['Xylene'].fillna(df['Xylene'].mean(), inplace=True)
df['PM2.5'].fillna(df['PM2.5'].mean(), inplace=True)
df['PM10'].fillna(df['PM10'].mean(), inplace=True)
df['NOx'].fillna(df['NOx'].mean(), inplace=True)
df['CO'].fillna(df['CO'].mean(), inplace=True)
df['Benzene'].fillna(df['Benzene'].mean(), inplace=True)
df['Toluene'].fillna(df['Toluene'].mean(), inplace=True)

In [ ]:
df.describe()

In [ ]:
df.to_csv('india-air-quality-2015-2020/Delhi/preprocess.csv', index=False)

# AQI Calculation

$$ {I_p} = {{{I_{Hi} - I_{Lo}} \over {BP_{Hi} - BP_{Lo}}}(C_p - BP_{Lo}) + I_{Lo}}$$

- $I_p$: the index for pollutant p
- $C_p$: the concentration of pollutant p
- $BP_{Hi}$: the concentration breakpoint that $ \geq C_p$
- $BP_{Lo}$: the concentration breakpoint that $ \leq C_p$
- $I_{Hi}$: the AQI value corresponding to the $BP_{Hi}$
- $I_{Lo}$: the AQI value corresponding to the $BP_{Lo}$


| AQI category (range)  | PM10 (24hr) | PM2.5 (24hr) | NO2 (24hr) | O3 (8hr) | CO (8hr) | SO2 (24hr) | NH3 (24hr) | Pb (24hr) |
|:----------------------|:-----------:|:------------:|:----------:|:--------:|:--------:|:----------:|:----------:|:---------:|
| Good (0–50)           |    0–50     |     0–30     |    0–40    |   0–50   |  0–1.0   |    0–40    |   0–200    |   0–0.5   |
| Satisfactory (51–100) |   51–100    |    31–60     |   41–80    |  51–100  | 1.1–2.0  |   41–80    |  201–400   |  0.5–1.0  |
| Moderate (101–200)    |   101–250   |    61–90     |   81–180   | 101–168  |  2.1–10  |   81–380   |  401–800   |  1.1–2.0  |
| Poor (201–300)        |   251–350   |    91–120    |  181–280   | 169–208  |  10–17   |  381–800   |  801–1200  |  2.1–3.0  |
| Very Poor (301–400)   |   351–430   |   121–250    |  281–400   | 209–748  |  17–34   |  801–1600  | 1200–1800  |  3.1–3.5  |
| Severe (401–500)      |    430+     |     250+     |    400+    |   748+   |   34+    |   1600+    |   1800+    |   3.5+    |

Suppose a monitor records a 24-hour average fine particle (PM2.5) concentration of 26.4 micrograms per cubic meter. The equation above results in an AQI of:

$$ {{\frac {50 - 0}{30 - 0}}(26.4-0)+0=83.2} $$

In [ ]:
df["PM10_24hr_avg"] = df["PM10"].rolling(window = 24, min_periods = 16).mean().values
df["PM2.5_24hr_avg"] = df["PM2.5"].rolling(window = 24, min_periods = 16).mean().values
df["NO2_24hr_avg"] = df["NO2"].rolling(window = 24, min_periods=16).mean().values
df["SO2_24hr_avg"] = df["SO2"].rolling(window = 24, min_periods = 16).mean().values
df["NOx_24hr_avg"] = df["NOx"].rolling(window = 24, min_periods = 16).mean().values
df["NH3_24hr_avg"] = df["NH3"].rolling(window = 24, min_periods = 16).mean().values
df["CO_8hr_max"] = df["CO"].rolling(window = 8, min_periods = 4).max().values
df["O3_8hr_max"] = df["O3"].rolling(window = 8, min_periods = 4).max().values

## PM2.5

In [ ]:
## PM2.5 Sub-Index calculation
def get_PM25_subindex(x):
    if x <= 30:
        return x * 50 / 30
    elif x <= 60:
        return 50 + (x - 30) * 50 / 30
    elif x <= 90:
        return 100 + (x - 60) * 100 / 30
    elif x <= 120:
        return 200 + (x - 90) * 100 / 30
    elif x <= 250:
        return 300 + (x - 120) * 100 / 130
    elif x > 250:
        return 400 + (x - 250) * 100 / 130
    else:
        return 0

df["PM2.5_SubIndex"] = df["PM2.5_24hr_avg"].apply(lambda x: get_PM25_subindex(x))


## PM10

In [ ]:
## PM10 Sub-Index calculation
def get_PM10_subindex(x):
    if x <= 50:
        return x
    elif x <= 100:
        return x
    elif x <= 250:
        return 100 + (x - 100) * 100 / 150
    elif x <= 350:
        return 200 + (x - 250)
    elif x <= 430:
        return 300 + (x - 350) * 100 / 80
    elif x > 430:
        return 400 + (x - 430) * 100 / 80
    else:
        return 0

df["PM10_SubIndex"] = df["PM10_24hr_avg"].apply(lambda x: get_PM10_subindex(x))


## SO2

In [ ]:
## SO2 Sub-Index calculation
def get_SO2_subindex(x):
    if x <= 40:
        return x * 50 / 40
    elif x <= 80:
        return 51 + (x - 40) * 50 / 40
    elif x <= 380:
        return 100 + (x - 80) * 100 / 300
    elif x <= 800:
        return 200 + (x - 380) * 100 / 420
    elif x <= 1600:
        return 300 + (x - 800) * 100 / 800
    elif x > 1600:
        return 400 + (x - 1600) * 100 / 800
    else:
        return 0


df["SO2_SubIndex"] = df["SO2_24hr_avg"].apply(lambda x: get_SO2_subindex(x))

## NOx

In [ ]:
## NOx Sub-Index calculation
def get_NOx_subindex(x):
    if x <= 40:
        return x * 50 / 40
    elif x <= 80:
        return 50 + (x - 40) * 50 / 40
    elif x <= 180:
        return 100 + (x - 80) * 100 / 100
    elif x <= 280:
        return 200 + (x - 180) * 100 / 100
    elif x <= 400:
        return 300 + (x - 280) * 100 / 120
    elif x > 400:
        return 400 + (x - 400) * 100 / 120
    else:
        return 0


df["NOx_SubIndex"] = df["NOx_24hr_avg"].apply(lambda x: get_NOx_subindex(x))

## NH3

In [ ]:
## NH3 Sub-Index calculation
def get_NH3_subindex(x):
    if x <= 200:
        return x * 50 / 200
    elif x <= 400:
        return 50 + (x - 200) * 50 / 200
    elif x <= 800:
        return 100 + (x - 400) * 100 / 400
    elif x <= 1200:
        return 200 + (x - 800) * 100 / 400
    elif x <= 1800:
        return 300 + (x - 1200) * 100 / 600
    elif x > 1800:
        return 400 + (x - 1800) * 100 / 600
    else:
        return 0


df["NH3_SubIndex"] = df["NH3_24hr_avg"].apply(lambda x: get_NH3_subindex(x))

## CO

In [ ]:
## CO Sub-Index calculation
def get_CO_subindex(x):
    if x <= 1:
        return x * 50 / 1
    elif x <= 2:
        return 50 + (x - 1) * 50 / 1
    elif x <= 10:
        return 100 + (x - 2) * 100 / 8
    elif x <= 17:
        return 200 + (x - 10) * 100 / 7
    elif x <= 34:
        return 300 + (x - 17) * 100 / 17
    elif x > 34:
        return 400 + (x - 34) * 100 / 17
    else:
        return 0


df["CO_SubIndex"] = df["CO_8hr_max"].apply(lambda x: get_CO_subindex(x))

## O3

In [ ]:
## O3 Sub-Index calculation
def get_O3_subindex(x):
    if x <= 50:
        return x * 50 / 50
    elif x <= 100:
        return 50 + (x - 50) * 50 / 50
    elif x <= 168:
        return 100 + (x - 100) * 100 / 68
    elif x <= 208:
        return 200 + (x - 168) * 100 / 40
    elif x <= 748:
        return 300 + (x - 208) * 100 / 539
    elif x > 748:
        return 400 + (x - 400) * 100 / 539
    else:
        return 0


df["O3_SubIndex"] = df["O3_8hr_max"].apply(lambda x: get_O3_subindex(x))

# AQI

Following The National Air Quality Index launched in New Delhi on 17/09/2014, under [Swachh Bharat Mission](https://en.wikipedia.org/wiki/Swachh_Bharat_Mission).     
There are six AQI categories, namely Good, Satisfactory, Moderate, Poor, Severe and Hazardous. The proposed AQI will consider eight pollutants (PM10, PM2.5, NO2, SO2, CO, O3, NH3, and Pb) for which short-term (up to 24-hourly averaging period) National Ambient Air Quality Standards are prescribed.[34] Based on the measured ambient concentrations, corresponding standards and likely health impact, a sub-index is calculated for each of these pollutants. The worst sub-index reflects overall AQI. Likely health impacts for different AQI categories and pollutants have also been suggested, with primary inputs from the medical experts in the group. The AQI values and corresponding ambient concentrations (health breakpoints) as well as associated likely health impacts for the identified eight pollutants are as follows:

In [ ]:
## AQI bucketing
def get_AQI_bucket(x):
    if x <= 50:
        return "Good"
    elif x <= 100:
        return "Satisfactory"
    elif x <= 200:
        return "Moderate"
    elif x <= 300:
        return "Poor"
    elif x <= 400:
        return "Very Poor"
    elif x > 400:
        return "Severe"
    else:
        return np.NaN


df["Checks"] = (
    (df["PM2.5_SubIndex"] > 0).astype(int)
    + (df["PM10_SubIndex"] > 0).astype(int)
    + (df["SO2_SubIndex"] > 0).astype(int)
    + (df["NOx_SubIndex"] > 0).astype(int)
    + (df["NH3_SubIndex"] > 0).astype(int)
    + (df["CO_SubIndex"] > 0).astype(int)
    + (df["O3_SubIndex"] > 0).astype(int)
)

df["AQI_calculated"] = round(
    df[
        [
            "PM2.5_SubIndex",
            "PM10_SubIndex",
            "SO2_SubIndex",
            "NOx_SubIndex",
            "NH3_SubIndex",
            "CO_SubIndex",
            "O3_SubIndex",
        ]
    ].max(axis=1)
)
df.loc[df["PM2.5_SubIndex"] + df["PM10_SubIndex"] <= 0, "AQI_calculated"] = np.NaN
df.loc[df.Checks < 3, "AQI_calculated"] = np.NaN

df["AQI_bucket_calculated"] = df["AQI_calculated"].apply(lambda x: get_AQI_bucket(x))
df[~df.AQI_calculated.isna()].head(13)

In [ ]:
df[~df.AQI_calculated.isna()].AQI_bucket_calculated.value_counts()

In [ ]:
df.to_csv('india-air-quality-2015-2020/aqi_calculated_test.csv', index=False)

In [ ]:
df.isnull().sum()

In [46]:
df1 = pd.read_csv('india-air-quality-2015-2020/Delhi/aqi.csv', parse_dates=["Datetime"])